In [68]:
from app.dotenv import base_dir, data_dir,golden_data_dir
# from app.model.model import  Model
# import os
import pandas as pd
import numpy as np
from pandas import DataFrame
# set the max columns to none
pd.set_option('display.max_columns', None)
# set the max columns to none
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
# print('dotenv params:', {os.getenv('PARAM1')})
print('base_dir:', base_dir)
print('data_dir:', data_dir)
print('golden_data_dir:', golden_data_dir)

# model = Model()
# print('model:', model)

import json

base_dir: /home/yishai/Desktop/ivrit_ai_llm/app
data_dir: /home/yishai/Desktop/ivrit_ai_llm/data
golden_data_dir: /home/yishai/Desktop/ivrit_ai_llm/golden_data


avg_logprob - the average log probability of the tokens in the transcription
estimating the confidence of the transcription. Lower values (more negative) indicate lower confidence, while values closer to zero indicate higher confidence in the transcription.
Values generally range between -1.0 and 0, below -0.5 is suspected below -1 is very suspected


no_speech_prob - probability is high (close to 1), it suggests that the model believes the segment is likely silence or background noise, rather than actual speech. This can be useful for filtering out non-speech parts.
Values range from 0 to 1, where 0 indicates a high likelihood that the segment contains speech, and 1 indicates a high probability that the segment contains no speech.

compression_ratio - measure of how much the output text has been "compressed" compared to the input audio in terms of information content. Higher values might indicate potential issues, such as long stretches of repeated text or overly verbose output.
Typically ranges from 1.0 to 2.5. A value of 1.0 means that the transcription is roughly equal in length to the original audio input, while higher values indicate more compression (shorter transcribed text relative to the audio duration).

Suggested Workflow:

Filter Low Confidence Segments: Identify segments with a low avg_logprob or a high no_speech_prob and prioritize these for refinement by the LLM.

Fix Repetitive or Garbled Transcriptions: Use the compression_ratio to detect where the transcribed text might be problematic (too verbose, repeated words) and ask the LLM to clean up these sections.

Time-based Post-Processing: Use the start and end timestamps to make sure the LLM processes segments in the right sequence, especially if you are breaking up the transcription for batch processing.

#### load df from file

1. add wanted attribute columns & remove nulls  
2. drop columns
3. remove formatting chracters


In [69]:
df = pd.read_csv(f'{golden_data_dir}/crowd-transcribe-v4-enriched.csv')
ind = ~df['attrs'].isnull()
df.loc[ind,'attrs'] = df.loc[ind,'attrs'].apply(lambda x: json.loads(x))
df = df.loc[((~df['sentence'].isnull())*(~df['orig_sentence'].isnull()))]
# verify nulls are dropped
df['sentence'].loc[(~(~df['sentence'].isnull())*(~df['orig_sentence'].isnull()))]


Series([], Name: sentence, dtype: object)

In [70]:
vals = {'avg_logprob':-0.5,'no_speech_prob':0.5,'compression_ratio':2.0}
inds = {}
for key,value in vals.items():
    df[key] = value
    # df.loc[0,'attrs']['segments'][0]['avg_logprob']
    df[key] = None
    df.loc[ind,key] = df.loc[ind,'attrs'].apply( lambda x:  x['segments'][0][key] )
    # data = df.loc[ind,key].loc[df.loc[ind,key]>value]
    inds[key] = (df.loc[ind,key].loc[df.loc[ind,key]>value].index,df.loc[ind,key].loc[df.loc[ind,key]<value].index)    
    print(len(inds[key][0]))


111858
1901
5307


drop columns

In [71]:
df.drop(columns=['attrs','is_retranscribe','uuid'],inplace=True)
df.columns

Index(['orig_sentence', 'sentence', 'source', 'episode', 'avg_logprob',
       'no_speech_prob', 'compression_ratio'],
      dtype='object')

remove chracters function

In [72]:
def remove_chars(df: DataFrame,columns,remove_ords =[],remove_chars =[],replace_char = '') -> DataFrame:

    def replace_chars_from_col(col,in_chars,replace_char = '',is_ord = False):
        for ch in in_chars:
            if is_ord:
                col = col.str.replace(chr(ch),replace_char)
            else:
                col = col.str.replace(ch,replace_char)
            
        # for ch in ['  ','-']:
        #     col = col.str.replace(ch,' ')
        return col

    
    for col in columns:
        df[col] = df[col].str.lower()
        df[col] = replace_chars_from_col(df[col].copy(),remove_ords,replace_char = replace_char,is_ord = True)

        df[col] = replace_chars_from_col(df[col].copy(),remove_chars,replace_char = replace_char,is_ord = False)
    return df

remove error characters

In [73]:





def print_existing_unique_chars(cols):
    
    def cols_unique_characters(cols,print_ord = False):
        uniq_chars = sorted(set(''.join(''.join(col) for col in cols)))
        if print_ord:
            return [ord(ch) for ch in uniq_chars]
        return uniq_chars
    
    uniq1 = cols_unique_characters(cols)
    uniq2 = cols_unique_characters(cols,print_ord = True)
    print ('existing chars and their ords') 
    print('\t'.join([str(x)+',' for x in uniq1]))
    print('\t'.join([str(x)+',' for x in uniq2]))
    

# error characters:

error_ords =[]
error_ords1 = [1470,1468,1465,1463,1462,1460,1099,1464,47,1523,	1524,	1575,	1576,	1578,	1582,	1585,	1603,	1606,	8211,	8212,	8230,	9834,	9835,	20126,	21478,	24555,	26126,	26449,	27794,	29978,	35201,	38590,	40636,	46988,	47673,	49324,	49900,	50508,	50520,	50612,	50880,	51032,	51060,	65533,]
error_ords += error_ords1
# # latin letters - might be usefull to keep
# remove_ords2 = [380,	945,	950,	956,	959,	960,	1072,	1073,	1074,	1075,	1076,	1077,	1080,	1082,	1083,	1086,	1087,	1088,	1089,	1091,	1096,	1100]
# remove_ords += remove_ords2

error_chars = ['\u202b','\u200f','\u200e','\n']

   
df = remove_chars(df.copy(),columns = ['orig_sentence','sentence'],
                  remove_ords = error_ords,remove_chars = error_chars,
                  replace_char = '')

df = remove_chars(df.copy(),columns = ['orig_sentence','sentence'],
                  remove_chars = ['  '], replace_char = ' ')        
print_existing_unique_chars([df['orig_sentence'].copy(),df['orig_sentence'].copy()])

existing chars and their ords
 ,	!,	",	$,	%,	&,	',	(,	),	+,	,,	-,	.,	0,	1,	2,	3,	4,	5,	6,	7,	8,	9,	:,	=,	?,	[,	],	a,	b,	c,	d,	e,	f,	g,	h,	i,	j,	k,	l,	m,	n,	o,	p,	q,	r,	s,	t,	u,	v,	w,	x,	y,	z,	ż,	α,	ζ,	μ,	ο,	π,	а,	б,	в,	г,	д,	е,	и,	к,	л,	о,	п,	р,	с,	у,	ш,	ь,	א,	ב,	ג,	ד,	ה,	ו,	ז,	ח,	ט,	י,	ך,	כ,	ל,	ם,	מ,	ן,	נ,	ס,	ע,	ף,	פ,	ץ,	צ,	ק,	ר,	ש,	ת,
32,	33,	34,	36,	37,	38,	39,	40,	41,	43,	44,	45,	46,	48,	49,	50,	51,	52,	53,	54,	55,	56,	57,	58,	61,	63,	91,	93,	97,	98,	99,	100,	101,	102,	103,	104,	105,	106,	107,	108,	109,	110,	111,	112,	113,	114,	115,	116,	117,	118,	119,	120,	121,	122,	380,	945,	950,	956,	959,	960,	1072,	1073,	1074,	1075,	1076,	1077,	1080,	1082,	1083,	1086,	1087,	1088,	1089,	1091,	1096,	1100,	1488,	1489,	1490,	1491,	1492,	1493,	1494,	1495,	1496,	1497,	1498,	1499,	1500,	1501,	1502,	1503,	1504,	1505,	1506,	1507,	1508,	1509,	1510,	1511,	1512,	1513,	1514,


remove empty columns

In [74]:

for col in ['orig_sentence','sentence']:
    ind = df[col].str.len()==0
    df = df.loc[~ind]   



#### baseline results - filter out samples without mistakes

WER - word error rate

calculate how well the hypothesis (orig_sentence) matches the correct reference (sentence)

WER = (S+D+I)/N

In [52]:
# from jiwer import wer

# reference = "the correct translation"
# hypothesis = "your corrected sentence"

# error_rate = wer(reference, hypothesis)
# print(f"Word Error Rate: {error_rate}")
import jiwer


transforms = jiwer.Compose(
    [
        jiwer.ExpandCommonEnglishContractions(),
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
)


wer = df.apply(lambda x: jiwer.wer(x['sentence'],x['orig_sentence'],
                                   truth_transform=transforms, hypothesis_transform=transforms,
                                   )
                     ,axis = 1)

In [10]:
LEN = len(df.loc[df['wer']==0.0])
print('zero wer:',LEN,' percentage:',LEN/len(df))
# zero wer: 78808  percentage: 0.50085479862978


df = df.loc[df['wer']>0.0]

zero wer: 78808  percentage: 0.50085479862978


In [75]:
df.reset_index(drop=True,inplace=True)
# for col in df.columns:
#     if 'float' in str(df[col].dtype):
#         df[col] = df[col].astype(float)
#     if 'int' in str(df[col].dtype):
#         df[col] = df[col].astype(int)
print([col for col in df.columns])


['orig_sentence', 'sentence', 'source', 'episode', 'avg_logprob', 'no_speech_prob', 'compression_ratio']


In [79]:

df.to_csv(f'{data_dir}/crowd-transcribe-v4-enriched-1.csv', index=False)
# df1 = pd.read_csv(f'{data_dir}/crowd-transcribe-v4-enriched-1.csv')


# ddf = df[~(df.isnull().sum(axis=1).astype(bool))].copy()
# ddf1 = df1[~(df1.isnull().sum(axis=1).astype(bool))].copy()
# if (ddf1==ddf).all().all():
    
#     print('df and df1 are equal')
#     del df1,ddf,ddf1#df,

In [14]:
print([type(df[col].loc[0]) for col in df.columns])
print([type(df1[col].loc[0]) for col in df.columns])


[<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'numpy.float64'>]
[<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>]
